In [ ]:
#Import the libraries
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import math as mt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.axes_divider import make_axes_area_auto_adjustable
from scipy.signal import chirp, find_peaks, peak_widths, find_peaks_cwt
import os
import cartopy.crs as ccrs
import matplotlib.ticker as mticker

In [ ]:
#Define diretory
datadir = '../Emission_processed_data'

#Define Year for processing
Year = 2020
#exemple file name: goes_data_emission_rate_amazon_definitive_box_2022_150_350.csv

data = sorted(glob.glob(datadir+'/*goes_data_emission_rate_amazon_definitive_box_'+str(Year)+'_150_350.csv'))
#print(data)

#All columns names in the data: sat,year,julian,hhmm,code,central_lat,central_lon,FRP(MW),N_FRP,FRE(MJ),RE(kg/s),ME(kg),
#CO_2(kg),sigma_CO_2(kg),CO(kg),sigma_CO(kg),CH4(kg),sigma_CH4(kg),mean_FRP,mean_temp
selected_columns = ['central_lat', 'central_lon','julian','ME(kg)','CO_2(kg)', 'sigma_CO_2(kg)','CO(kg)', 'sigma_CO(kg)','CH4(kg)', 'sigma_CH4(kg)']

# Read the CSV file extracting just the selected columns
emission_data = pd.read_csv(data[0], usecols=selected_columns)

In [ ]:
#Function to plot a graph of the daily emission of TPM
def TPM_daily_emission(dt_emission, filename):
    #List the data file header
    header = list(dt_emission)

    #Extract the columns of the file
    pos1 = header.index('julian')
    column1 = dt_emission.iloc[:, pos1]
    julian = np.array(column1.dropna(), dtype=pd.Series)

    pos3 = header.index('ME(kg)')
    column3 = dt_emission.iloc[:, pos3]
    TPM = np.array(column3.dropna(), dtype=pd.Series)

    #Prepare the figure layout
    plt.clf()
    plt.title(str(Year))
    plt.ylabel(r'TPM (Tg)')
    plt.xlabel('Days of the year')

    #The TPM emission is calculated every 10 minutes, so it is necessary to sum the results over day period
    days = np.unique(julian)
    matriz = []
    for i in range(np.min(days),np.max(days)+1):
        # print(i)
        index = np.where(julian ==i)
        daily_sum = np.sum(TPM[index])
        matriz = np.append(matriz,daily_sum)

    #Collect the max emission day through the matrix of daily emission
    max_emission_day_index = np.where(matriz == np.max(matriz))
    max_emission_day =days[max_emission_day_index]

    #Configure the plot
    plt.ylim(0, np.max(matriz)*1.1/ 1e9)
    #plt.ticklabel_format(axis="y",)
    plt.plot(days, matriz/ 1e9, 'o', markersize=2, color='black')
    Total = 'Total\nemission: {:.2e} Tg'.format(
        np.sum(TPM[TPM > 0])*1e-9)
    plt.legend([Total], loc='best')
    #plt save
    plt.savefig(filename+'.png', dpi=200)

    #Return the max emission day. It will be used on other functions
    return max_emission_day[0]

In [ ]:
#Function to plot a graph of the daily emission of CO2
def CO2_stats_v2(dt_emission, filename):
    #List the data file header
    header = list(dt_emission)

    #Extract the columns of the file
    pos1 = header.index('julian')
    column1 = dt_emission.iloc[:, pos1]
    julian = np.array(column1.dropna(), dtype=pd.Series)

    pos3 = header.index('CO_2(kg)')
    column3 = dt_emission.iloc[:, pos3]
    co2 = np.array(column3.dropna(), dtype=pd.Series)

    pos4 = header.index('sigma_CO_2(kg)')
    column4 = dt_emission.iloc[:, pos4]
    sigma_co2 = np.array(column4.dropna(), dtype=pd.Series)


    #Prepare the figure layoutO
    pyplot.clf()
    pyplot.title(str(Year))
    pyplot.ylabel('CO_2(kg)')
    pyplot.xlabel('Julian days')

    #The CO2 emission is calculated every 10 minutes, so it is necessary to sum the results over day period
    days = np.unique(julian)
    matriz = np.zeros((len(days), 2))
    for i in range(0,len(days)):
        index = np.where(julian ==days[i])

        # Convert the data and uncertainties to Uncertainties objects
        u_data = unp.uarray(co2[index], sigma_co2[index])

        # Calculate the total sum with automatic uncertainty propagation
        u_sum_total = np.sum(u_data)

        # Extract the resulting value and uncertainty
        sum_total_value = unp.nominal_values(u_sum_total)
        sum_total_uncertainty = unp.std_devs(u_sum_total)

        matriz[i,0] = sum_total_value
        matriz[i,1] = sum_total_uncertainty


    #Configure the plot
    pyplot.ylim(0, np.max(matriz[:, 0])*1.1)
    pyplot.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    pyplot.errorbar(days,  matriz[:, 0], yerr=matriz[:, 1], ls='none',color='black',
                    marker='o', markersize=2, ecolor='red')
    Total = 'Total\nemission: {:.2e} Tg'.format(np.sum(co2[co2 > 0])*1e-9)
    pyplot.legend([Total], loc='best')

    #And save
    pyplot.savefig(filename+'.png', dpi=200)


    return

In [ ]:
#Function to plot the daily(max emission day) emission of TPM and CO2 on a grid
#The grid cell has a 0.5°x0.5° of area
#The region of the grid was determined on previous codes that delimited the Amazon region of interest
def TPM_and_CO2_mapping_v1(dados_emission, filename,day_emission_max):
    #List the data file header
    header = list(dados_emission)

    #Extract the columns of the file
    pos1 = header.index('julian')
    column1 = dados_emission.iloc[:, pos1]
    julian = np.array(column1.dropna(), dtype=pd.Series)

    pos3 = header.index('ME(kg)')
    column3 = dados_emission.iloc[:, pos3]
    TPM = np.array(column3.dropna(), dtype=pd.Series)

    pos2 = header.index('central_lat')
    column2 = dados_emission.iloc[:, pos2]
    lat = np.array(column2.dropna(), dtype=pd.Series)

    pos4 = header.index('central_lon')
    column4 = dados_emission.iloc[:, pos4]
    lon = np.array(column4.dropna(), dtype=pd.Series)

    pos3 = header.index('CO_2(kg)')
    column3 = dados_emission.iloc[:, pos3]
    CO2 = np.array(column3.dropna(), dtype=pd.Series)

    #Extract the lats and lons of the region without repetions
    lats = np.unique(lat)
    lons = np.unique(lon)
    lats = lats[::-1] #It is necessary to invert the order of the latitudes

    #Configure the matrices based on the lats and lons
    matrix_TPM = np.zeros((len(lats), len(lons)), dtype='float64')
    matrix_CO2 = np.zeros((len(lats), len(lons)), dtype='float64')

    #Loop through the lats and lons, for each lat, lon and day(in this case only the max emission day)
    #save on the cell of the matrices the emission that correspond to each lat and lon
    for k in range(0, len(lats)):
        Progress = 'Progress: {:.1f}lat/{:.1f}lats'.format(k, len(lats))
        print(Progress)
        for n in range(0, len(lons)):
            index = np.where((lat == lats[k]) & (lon == lons[n]) & (julian == day_emission_max))
            matrix_TPM[k, n] = np.sum(TPM[index])*1e-3
            matrix_CO2[k, n] = np.sum(CO2[index])*1e-3

    #Configure the plot
    pyplot.clf()
    fig, ax = pyplot.subplots(figsize=(6.5, 2.5))
    extend = np.min(lons), np.max(lons),  np.min(lats), np.max(lats)
    #Set the min and max based on the percetile to a better visualization of
    #the distribuiton of the emission over the region
    im = ax.imshow(matrix_TPM, cmap='inferno', extent=extend,vmin=np.percentile(matrix_TPM,30),vmax=np.percentile(matrix_TPM,99))
    im2 = ax.imshow(matrix_CO2, cmap='inferno', extent=extend,vmin=np.percentile(matrix_CO2,30),vmax=np.percentile(matrix_CO2,95))

    ax.set_xlabel('lon')
    ax.set_ylabel('lat')
    ax.set_title(str(Year))
    Label = "TPM(Ton) day {:.0f}".format(day_emission_max)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cax2 = divider.append_axes("right", size="0.05%", pad=0.65)

    #Configure the colorbars
    vmin_tpm=np.percentile(matrix_TPM,30)
    vmax_tpm=np.percentile(matrix_TPM,99)
    tick_interval = (vmax_tpm-vmin_tpm)/8 # Interval between the values of the colorbar

    cb = pyplot.colorbar(im, cax=cax, format='%.1e')
    cb.set_label(Label, fontsize=9)
    cb.set_ticks(np.arange(vmin_tpm, vmax_tpm , tick_interval))
    cb.ax.tick_params(labelsize=6)
    Label2 = "CO2(Ton) day {:.0f}".format(day_emission_max)

    vmin=np.percentile(matrix_CO2,30)
    vmax=np.percentile(matrix_CO2,95)
    tick_interval = (vmax-vmin)/8 # Interval between the values of the colorbar

    cb2 = pyplot.colorbar(im2, cax=cax2, format='%.1e')
    cb2.set_label(Label2, fontsize=9)
    cb2.set_ticks(np.arange(vmin, vmax , tick_interval))
    cb2.ax.tick_params(labelsize=6)

    #Save the figure
    fig.savefig(filename+'.png', dpi=200)

    return

In [ ]:
#Function to plot a graph of the daily emission of CO
def CO_stats_v2(dados_emission, filename):
    #List the data file header
    header = list(dados_emission)

    #Extract the columns of the file
    pos1 = header.index('julian')
    column1 = dados_emission.iloc[:, pos1]
    julian = np.array(column1.dropna(), dtype=pd.Series)

    pos3 = header.index('CO(kg)')
    column3 = dados_emission.iloc[:, pos3]
    co = np.array(column3.dropna(), dtype=pd.Series)

    pos4 = header.index('sigma_CO(kg)')
    column4 = dados_emission.iloc[:, pos4]
    sigma_co = np.array(column4.dropna(), dtype=pd.Series)


    #Prepare the figure layout
    pyplot.clf()
    pyplot.title(str(Year))
    pyplot.ylabel('CO(kg)')
    pyplot.xlabel('Dias julianos')

    #The CO emission is calculated every 10 minutes, so it is necessary to sum the results over day period
    dias = np.unique(julian)
    matriz = np.zeros((len(dias), 2))
    for i in range(0,len(dias)):
        index = np.where(julian ==dias[i])

        # Convert the data and uncertainties to Uncertainties objects
        u_data = unp.uarray(co[index], sigma_co[index])

        # Calculate the total sum with automatic uncertainty propagation
        u_sum_total = np.sum(u_data)

        # Extract the resulting value and uncertainty
        sum_total_value = unp.nominal_values(u_sum_total)
        sum_total_uncertainty = unp.std_devs(u_sum_total)

        matriz[i,0] = sum_total_value
        matriz[i,1] = sum_total_uncertainty

    #Configure the plot
    pyplot.ylim(0, np.max(matriz[:, 0])*1.1)
    pyplot.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    pyplot.errorbar(dias,  matriz[:, 0], yerr=matriz[:, 1], ls='none',color='black',
                    marker='o', markersize=2, ecolor='red')
    Total = 'Emissão\ntotal: {:.2e} Tg'.format(np.sum(co[co > 0])*1e-9)
    pyplot.legend([Total], loc='best')

    #And save
    pyplot.savefig(filename+'.png', dpi=200)

    return

In [ ]:
#Function to plot a graph of the daily emission of CH4
def CH4_stats_v2(dados_emission, filename):
    #List the data file header
    header = list(dados_emission)

    #Extract the columns of the file
    pos1 = header.index('julian')
    column1 = dados_emission.iloc[:, pos1]
    julian = np.array(column1.dropna(), dtype=pd.Series)

    pos3 = header.index('CH4(kg)')
    column3 = dados_emission.iloc[:, pos3]
    ch4 = np.array(column3.dropna(), dtype=pd.Series)

    pos4 = header.index('sigma_CH4(kg)')
    column4 = dados_emission.iloc[:, pos4]
    sigma_ch4 = np.array(column4.dropna(), dtype=pd.Series)

    #Prepare the figure layout
    pyplot.clf()
    pyplot.title(str(Year))
    pyplot.ylabel('CH4(kg)')
    pyplot.xlabel('Dias julianos')

    #The CH4 emission is calculated every 10 minutes, so it is necessary to sum the results over day period
    dias = np.unique(julian)
    matriz = np.zeros((len(dias), 2))
    for i in range(0,len(dias)):
        index = np.where(julian ==dias[i])


        # Convert the data and uncertainties to Uncertainties objects
        u_data = unp.uarray(ch4[index], sigma_ch4[index])

        # Calculate the total sum with automatic uncertainty propagation
        u_sum_total = np.sum(u_data)

        # Extract the resulting value and uncertainty
        sum_total_value = unp.nominal_values(u_sum_total)
        sum_total_uncertainty = unp.std_devs(u_sum_total)

        matriz[i,0] = sum_total_value
        matriz[i,1] = sum_total_uncertainty

    #Configure the plot
    pyplot.ylim(0, np.max(matriz[:, 0])*1.1)
    pyplot.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    pyplot.errorbar(dias,  matriz[:, 0], yerr=matriz[:, 1], ls='none',color='black',
                    marker='o', markersize=2, ecolor='red')
    Total = 'Emissão\ntotal: {:.2e} Tg'.format(np.sum(ch4[ch4 > 0])*1e-9)
    pyplot.legend([Total], loc='best')

    #And save
    pyplot.savefig(filename+'.png', dpi=200)

    return

In [ ]:
dia_emissao_max = 247
dia_emissao_max = TPM_daily_emission(emission_data,'TPM_emission_distribuition_amazon_definitive_box_'+str(Year))
#TPM_and_CO2_mapping_v1(emission_data,'TPM_and_C02_emission_mapping_amazon_definitive_box_'+str(Year)+'_total',dia_emissao_max)

In [ ]:
day_emission_max=232
day_emission_max

In [ ]:
dados_emission = emission_data
filename = 'TPM_and_C02_emission_mapping_amazon_definitive_box_'+str(Year)+'_total'
#day_emission_max = 247
#Function to plot the daily(max emission day) emission of TPM and CO2 on a grid
#The grid cell has a 0.5°x0.5° of area
#The region of the grid was determined on previous codes that delimited the Amazon region of interest

#List the data file header
header = list(dados_emission)

#Extract the columns of the file
pos1 = header.index('julian')
column1 = dados_emission.iloc[:, pos1]
julian = np.array(column1.dropna(), dtype=pd.Series)

pos3 = header.index('ME(kg)')
column3 = dados_emission.iloc[:, pos3]
TPM = np.array(column3.dropna(), dtype=pd.Series)

pos2 = header.index('central_lat')
column2 = dados_emission.iloc[:, pos2]
lat = np.array(column2.dropna(), dtype=pd.Series)

pos4 = header.index('central_lon')
column4 = dados_emission.iloc[:, pos4]
lon = np.array(column4.dropna(), dtype=pd.Series)

pos3 = header.index('CO_2(kg)')
column3 = dados_emission.iloc[:, pos3]
CO2 = np.array(column3.dropna(), dtype=pd.Series)

#Extract the lats and lons of the region without repetions
lats = np.unique(lat)
lons = np.unique(lon)
lats = lats[::-1] #It is necessary to invert the order of the latitudes

#Configure the matrices based on the lats and lons
matrix_TPM = np.zeros((len(lats), len(lons)), dtype='float64')
matrix_CO2 = np.zeros((len(lats), len(lons)), dtype='float64')

#Loop through the lats and lons, for each lat, lon and day(in this case only the max emission day)
#save on the cell of the matrices the emission that correspond to each lat and lon
for k in range(0, len(lats)):
    Progress = 'Progress: {:.1f}lat/{:.1f}lats'.format(k, len(lats))
    print(Progress)
    for n in range(0, len(lons)):
        index = np.where((lat == lats[k]) & (lon == lons[n]) & (julian == day_emission_max))
        matrix_TPM[k, n] = np.sum(TPM[index])*1e-3
        matrix_CO2[k, n] = np.sum(CO2[index])*1e-3

In [ ]:
# Define the function to format latitude and longitude
def format_latitude(y, _):
    return f"{abs(y):.0f}°S" if y < 0 else f"{y:.0f}°N"

def format_longitude(x, _):
    return f"{abs(x):.0f}°W" if x < 0 else f"{x:.0f}°E"

In [ ]:
#Configure the plot
#plt.clf()
fig, ax = plt.subplots(figsize=(10, 3))
extend = np.min(lons), np.max(lons),  np.min(lats), np.max(lats)
#Set the min and max based on the percetile to a better visualization of
#the distribuiton of the emission over the region
im = ax.imshow(matrix_TPM, cmap='inferno', extent=extend,vmin=np.percentile(matrix_TPM,30),vmax=np.percentile(matrix_TPM,99))
im2 = ax.imshow(matrix_CO2, cmap='inferno', extent=extend,vmin=np.percentile(matrix_CO2,30),vmax=np.percentile(matrix_CO2,95))

#ax.set_xlabel('lon')
#ax.set_ylabel('lat')
ax.set_title(str(Year), fontsize=10)
# Apply custom formatting to ticks
ax.xaxis.set_major_formatter(mticker.FuncFormatter(format_longitude))
ax.yaxis.set_major_formatter(mticker.FuncFormatter(format_latitude))

# Add bold text in the upper-left corner
ax.text(
    0.01, 0.99,               # Coordinates (relative to axes: 0=left/bottom, 1=right/top)
    'A)',        # Text content
    transform=ax.transAxes,   # Use axes-relative coordinates
    fontsize=12,              # Font size
    fontweight='bold',
    color = 'white', # Bold text
    ha='left',                # Horizontal alignment
    va='top',                 # Vertical alignment
)

Label = "TPM(Ton) on day {:.0f}".format(day_emission_max)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0.05)
cax2 = divider.append_axes("right", size="0.05%", pad=0.85)

#Configure the colorbars
vmin_tpm=np.percentile(matrix_TPM,30)
vmax_tpm=np.percentile(matrix_TPM,99)
tick_interval = 500 # Interval between the values of the colorbar

cb = plt.colorbar(im, cax=cax, format='%.1e')
cb.set_label(Label, fontsize=10)
cb.set_ticks(np.arange(vmin_tpm, vmax_tpm , tick_interval))
cb.ax.tick_params(labelsize=9)
Label2 = "CO2(Ton) on day {:.0f}".format(day_emission_max)

vmin=np.percentile(matrix_CO2,30)
vmax=np.percentile(matrix_CO2,99)
tick_interval = 100000 # Interval between the values of the colorbar

cb2 = plt.colorbar(im2, cax=cax2, format='%.1e')
cb2.set_label(Label2, fontsize=10)
cb2.set_ticks(np.arange(vmin, vmax , tick_interval))
cb2.ax.tick_params(labelsize=9)

#Save the figure
plt.tight_layout()
#plt.show()
plt.savefig(filename+'.png', dpi=300)

In [ ]:
#Configure the plot
#plt.clf()
fig, ax = plt.subplots(figsize=(4, 4))
#extend = np.min(lons), np.max(lons),  np.min(lats), np.max(lats)
#6.0° to 9.0° S, 56.0° to 58.0° W
minlon, maxlon, minlat, maxlat = -57, -54, -9, -6  # Amazon small box
extend = minlon, maxlon,  minlat, maxlat
#Set the min and max based on the percetile to a better visualization of
#the distribuiton of the emission over the region
im = ax.imshow(matrix_TPM, cmap='inferno', extent=extend,vmin=np.percentile(matrix_TPM,30),vmax=np.percentile(matrix_TPM,99))
#im2 = ax.imshow(matrix_CO2, cmap='inferno', extent=extend,vmin=np.percentile(matrix_CO2,30),vmax=np.percentile(matrix_CO2,95))

#ax.set_xlabel('lon')
#ax.set_ylabel('lat')
#ax.set_title(str(Year), fontsize=10)
# Apply custom formatting to ticks
ax.set_xticks([-57, -56, -55, -54])
ax.set_yticks([-9, -8, -7, -6])
ax.xaxis.set_major_formatter(mticker.FuncFormatter(format_longitude))
#ax.tick_params()
ax.yaxis.set_major_formatter(mticker.FuncFormatter(format_latitude))


Label = "Emitted daily TPM (ton)"
ax.set_title(Label, fontsize=10)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="7%", pad=0.05)
#cax2 = divider.append_axes("right", size="7%", pad=0.05)

#Configure the colorbars
vmin_tpm=-100
vmax_tpm=np.percentile(matrix_TPM,99)
tick_interval = 200 # Interval between the values of the colorbar

cb = plt.colorbar(im, cax=cax, format='%.0e')
#cb.set_label(Label, fontsize=10)
cb.set_ticks(np.arange(vmin_tpm, vmax_tpm , tick_interval))
cb.ax.tick_params(labelsize=9)
#Label2 = "CO2(Ton) on day {:.0f}".format(day_emission_max)

#vmin=np.percentile(matrix_CO2,30)
#vmax=np.percentile(matrix_CO2,95)
#tick_interval = 20000 # Interval between the values of the colorbar

#cb2 = plt.colorbar(im2, cax=cax2, format='%.1e')
#cb2.set_label(Label2, fontsize=10)
#cb2.set_ticks(np.arange(vmin, vmax , tick_interval))
#cb2.ax.tick_params(labelsize=9)

#Save the figure
plt.tight_layout()
#plt.show()
plt.savefig(filename+'.png', dpi=300)

In [ ]:
dados_emission = emission_data
filename_small = 'TPM_and_C02_emission_mapping_amazon_small_box_'+str(Year)+'_total'
minlon, maxlon, minlat, maxlat = -57, -54, -9, -6  # Amazon small box
#day_emission_max = 247
#Function to plot the daily(max emission day) emission of TPM and CO2 on a grid
#The grid cell has a 0.5°x0.5° of area
#The region of the grid was determined on previous codes that delimited the Amazon region of interest

#List the data file header
header = list(dados_emission)

#Extract the columns of the file
pos1 = header.index('julian')
column1 = dados_emission.iloc[:, pos1]
julian = np.array(column1.dropna(), dtype=pd.Series)

pos3 = header.index('ME(kg)')
column3 = dados_emission.iloc[:, pos3]
TPM = np.array(column3.dropna(), dtype=pd.Series)

pos2 = header.index('central_lat')
column2 = dados_emission.iloc[:, pos2]
lat = np.array(column2.dropna(), dtype=pd.Series)

pos4 = header.index('central_lon')
column4 = dados_emission.iloc[:, pos4]
lon = np.array(column4.dropna(), dtype=pd.Series)

pos3 = header.index('CO_2(kg)')
column3 = dados_emission.iloc[:, pos3]
CO2 = np.array(column3.dropna(), dtype=pd.Series)

#Extract the lats and lons of the region without repetions
lats = np.unique(lat)
lons = np.unique(lon)
lats = lats[::-1] #It is necessary to invert the order of the latitudes

lats = lats[(lats >= minlat) & (lats <= maxlat)]
lons = lons[(lons >= minlon) & (lons <= maxlon)]

#Configure the matrices based on the lats and lons
matrix_TPM = np.zeros((len(lats), len(lons)), dtype='float64')

#Loop through the lats and lons, for each lat, lon and day(in this case only the max emission day)
#save on the cell of the matrices the emission that correspond to each lat and lon
for k in range(0, len(lats)):
    Progress = 'Progress: {:.1f}lat/{:.1f}lats'.format(k, len(lats))
    print(Progress)
    for n in range(0, len(lons)):
        index = np.where((lat == lats[k]) & (lon == lons[n]) & (julian == day_emission_max))
        matrix_TPM[k, n] = np.sum(TPM[index])*1e-3

In [ ]:
#Configure the plot
#plt.clf()
fig, ax = plt.subplots(figsize=(4, 4))
#extend = np.min(lons), np.max(lons),  np.min(lats), np.max(lats)
#6.0° to 9.0° S, 56.0° to 58.0° W
minlon, maxlon, minlat, maxlat = -57, -54, -9, -6  # Amazon small box
extend = minlon, maxlon,  minlat, maxlat
#Set the min and max based on the percetile to a better visualization of
#the distribuiton of the emission over the region
im = ax.imshow(matrix_TPM, cmap='inferno', extent=extend,vmin=-200,vmax=np.percentile(matrix_TPM,100))
#im2 = ax.imshow(matrix_CO2, cmap='inferno', extent=extend,vmin=np.percentile(matrix_CO2,30),vmax=np.percentile(matrix_CO2,95))

#ax.set_xlabel('lon')
#ax.set_ylabel('lat')
#ax.set_title(str(Year), fontsize=10)
# Apply custom formatting to ticks
ax.set_xticks([-57, -56, -55, -54])
ax.set_yticks([-9, -8, -7, -6])
ax.xaxis.set_major_formatter(mticker.FuncFormatter(format_longitude))
#ax.tick_params()
ax.yaxis.set_major_formatter(mticker.FuncFormatter(format_latitude))


Label = "Emitted daily TPM (ton)"
ax.set_title(Label, fontsize=10)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="7%", pad=0.05)
#cax2 = divider.append_axes("right", size="7%", pad=0.05)

#Configure the colorbars
vmin_tpm=-200
vmax_tpm=np.percentile(matrix_TPM,100)
tick_interval = 200 # Interval between the values of the colorbar

cb = plt.colorbar(im, cax=cax, format='%.0e')
#cb.set_label(Label, fontsize=10)
cb.set_ticks(np.arange(vmin_tpm, vmax_tpm , tick_interval))
cb.ax.tick_params(labelsize=9)
#Label2 = "CO2(Ton) on day {:.0f}".format(day_emission_max)

#vmin=np.percentile(matrix_CO2,30)
#vmax=np.percentile(matrix_CO2,95)
#tick_interval = 20000 # Interval between the values of the colorbar

#cb2 = plt.colorbar(im2, cax=cax2, format='%.1e')
#cb2.set_label(Label2, fontsize=10)
#cb2.set_ticks(np.arange(vmin, vmax , tick_interval))
#cb2.ax.tick_params(labelsize=9)

#Save the figure
plt.tight_layout()
#plt.show()
plt.savefig(filename_small+'.png', dpi=300)